In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('dark_background')

In [3]:
df=pd.read_csv('../input/zomato-bangalore-restaurants/zomato.csv')
df.head()

In [4]:
df.shape

In [5]:
df.columns

In [6]:
df.info()

In [7]:
df.drop_duplicates(inplace=True)
df.shape

In [8]:
df['rate'].unique()

In [4]:
def handlerate(value):
    if(value=='NEW' or value=='-'):
        return np.nan
    else:
        value=str(value).split('/')
        value=value[0]
        return float(value)

df['rate']=df['rate'].apply(handlerate)
df['rate'].head()

In [5]:
df['rate'].fillna(df['rate'].mean(),inplace=True)
df['rate'].isnull().sum()

In [17]:
df.info()

In [18]:
df.dropna(inplace=True)
df.head()

In [7]:
df.rename(columns={'approx_cost(for two people)':'Cost2plates','listed_in(type)':'Type'}, inplace=True)
df.head()

In [20]:
df=df.drop(['listed_in(city)'],axis=1)

In [21]:
df.head()

In [23]:
df=df.drop(['url','address','phone','menu_item','dish_liked','reviews_list'],axis=1)
df.head()

In [10]:
def handlecomma(value):
    value=str(value)
    if ',' in value:
        value=value.replace(',','')
        return float(value)
    else :
        return float(value)
    
df['Cost2plates']=df['Cost2plates'].apply(handlecomma)
df['Cost2plates'].unique()

In [26]:
df['rest_type'].value_counts()

In [13]:
rest_types=df['rest_type'].value_counts(ascending=False)
rest_types

In [14]:
rest_types_lessthan1000=rest_types[rest_types<1000]
rest_types_lessthan1000

In [15]:
def handle_rest_type(value):
    if(value in rest_types_lessthan1000):
        return 'others'
    else:
        return value
    
df['rest_type']=df['rest_type'].apply(handle_rest_type)
df['rest_type'].value_counts()

In [16]:
location=df['location'].value_counts(ascending=False)
location_lessthan300=location[location<300]

def handle_location(value):
    if(value in location_lessthan300):
        return 'others'
    else:
        return value

df['location']=df['location'].apply(handle_location)
df['location'].value_counts()

In [17]:
cuisines=df['cuisines'].value_counts(ascending=False)
cuisines_lessthan100=cuisines[cuisines<100]

def handle_cuisines(value):
    if(value in cuisines_lessthan100):
        return 'others'
    else:
        return value
    
df['cuisines']=df['cuisines'].apply(handle_cuisines)
df['cuisines'].value_counts()

In [5]:
plt.figure(figsize=(16,10))
ax=sns.countplot(df['location'])
plt.xticks(rotation=90)

In [9]:
plt.figure(figsize=(6,6))
sns.countplot(df['online_order'],palette='inferno')

In [11]:
plt.figure(figsize=(6,6))
sns.countplot(df['book_table'],palette='rainbow')

In [36]:
plt.figure(figsize=(6,6))
sns.boxplot(x='online_order',y='rate',data=df)

In [15]:
plt.figure(figsize=(6,6))
sns.boxplot(x='book_table',y='rate',data=df)

# Visualizing Online Order Facility

In [16]:
df1=df.groupby(['location','online_order'])['name'].count()
df1.to_csv('location_online.csv')
df1=pd.read_csv('location_online.csv')
df1=pd.pivot_table(df1,values=None,index=['location'],columns=['online_order'],fill_value=0,aggfunc=np.sum)
df1

In [17]:
df1.plot(kind='bar',figsize=(15,8))


In [19]:
df2=df.groupby(['location','book_table'])['name'].count()
df2.to_csv('location_booktable.csv')
df2=pd.read_csv('location_booktable.csv')
df2=pd.pivot_table(df2,values=None,index=['location'],columns=['book_table'],fill_value=0,aggfunc=np.sum)
df2

In [20]:
df2.plot(kind='bar',figsize=(15,8))

In [38]:
plt.figure(figsize=(14,8))
sns.boxplot(x='Type',y='rate',data=df)

In [18]:
df3=df.groupby(['location','Type'])['name'].count()
df3.to_csv('location_Type.csv')
df3=pd.read_csv('location_Type.csv')
df3=pd.pivot_table(df3,values=None,index=['location'],columns=['Type'],fill_value=0,aggfunc=np.sum)
df3

In [20]:
df3.plot(kind='bar', figsize=(36,8))

In [22]:
df4=df[['location','votes']]
df.drop_duplicates()
df5=df.groupby(['location'])['votes'].sum()
df5=df5.to_frame()
df5=df5.sort_values('votes', ascending=False)
df5.head()

In [23]:
plt.figure(figsize=(15,8))
sns.barplot(df5.index,df5['votes'])
plt.xticks(rotation=90)

In [24]:
df6=df[['cuisines','votes']]
df6.drop_duplicates()
df7=df6.groupby(['cuisines'])['votes'].sum()
df7=df7.to_frame()
df7=df7.sort_values('votes',ascending=False)
df7.head()

In [26]:
df7=df7.iloc[1:,:]
df7.head()

In [27]:
plt.figure(figsize=(15,8))
sns.barplot(df7.index,df7['votes'])
plt.xticks(rotation=90)